# 1. 方差阈值法
1. 方差阈值法是一种简单的过滤方法，它通过移除方差小于给定阈值的特征来减少数据集的维度。方差低的特征通常对模型的区分能力贡献不大，特别是在分类任务中。原理方差是衡量数据分布离散程度的一个指标。对于离散程度非常小的特征（即方差接近于0），在分类问题中几乎不提供任何有用的信息。因此，这些特征可以被安全地移除。核心公式其中， 是第  个特征的方差， 是第  个样本在特征  上的取值， 是特征  的均值。推导：1. 均值计算：首先计算特征  的均值。
2. 方差计算：接着，计算每个样本值与均值的偏差的平方，并求平均值。
3. 方差比较：如果  小于设定的阈值，则剔除特征 。
4. 阈值设置：阈值可以根据经验设定或通过交叉验证调整，通常选择一个较小的正数。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

# 设置随机种子以保证结果可重复
np.random.seed(42)

# 生成虚拟数据集
n_samples = 1000
n_features = 5

# 数据分布：0~1均匀分布
X = np.random.rand(n_samples, n_features)

# 人为地将某些特征的方差设置为接近0（即无变化）
X[:, 1] = 0.5  # 第二个特征方差为0
X[:, 3] = X[:, 3] * 0.01  # 第四个特征方差非常小

# 创建DataFrame以便查看数据
columns = [f'Feature_{i+1}' for i in range(n_features)]
df = pd.DataFrame(X, columns=columns)

# 方差阈值法
threshold = 0.01  # 设置阈值
selector = VarianceThreshold(threshold=threshold)
X_selected = selector.fit_transform(X)

# 保留的特征
retained_features = df.columns[selector.get_support()]
dropped_features = df.columns[~selector.get_support()]

# PCA降维到2D以便可视化
pca = PCA(n_components=2)
X_pca_before = pca.fit_transform(X)
X_pca_after = pca.fit_transform(X_selected)

# 创建图形
fig, axes = plt.subplots(2, 2, figsize=(14, 10), constrained_layout=True)
fig.suptitle('Variance Threshold Feature Selection', fontsize=16)

# 原始数据的散点图 (PCA降维到2D)
axes[0, 0].scatter(X_pca_before[:, 0], X_pca_before[:, 1], c='blue', edgecolor='k', s=50, alpha=0.7)
axes[0, 0].set_title('Original Data (PCA 2D)', fontsize=14)
axes[0, 0].set_xlabel('PCA Component 1')
axes[0, 0].set_ylabel('PCA Component 2')

# 方差阈值法后的数据散点图 (PCA降维到2D)
axes[0, 1].scatter(X_pca_after[:, 0], X_pca_after[:, 1], c='green', edgecolor='k', s=50, alpha=0.7)
axes[0, 1].set_title('After Variance Threshold (PCA 2D)', fontsize=14)
axes[0, 1].set_xlabel('PCA Component 1')
axes[0, 1].set_ylabel('PCA Component 2')

# 原始数据的热力图
im1 = axes[1, 0].imshow(np.corrcoef(X.T), cmap='viridis', aspect='auto')
axes[1, 0].set_title('Original Data Correlation Heatmap', fontsize=14)
axes[1, 0].set_xticks(np.arange(n_features))
axes[1, 0].set_yticks(np.arange(n_features))
axes[1, 0].set_xticklabels(columns, rotation=45)
axes[1, 0].set_yticklabels(columns)
fig.colorbar(im1, ax=axes[1, 0])

# 方差阈值法后的数据热力图
im2 = axes[1, 1].imshow(np.corrcoef(X_selected.T), cmap='viridis', aspect='auto')
axes[1, 1].set_title('After Variance Threshold Correlation Heatmap', fontsize=14)
axes[1, 1].set_xticks(np.arange(len(retained_features)))
axes[1, 1].set_yticks(np.arange(len(retained_features)))
axes[1, 1].set_xticklabels(retained_features, rotation=45)
axes[1, 1].set_yticklabels(retained_features)
fig.colorbar(im2, ax=axes[1, 1])

# 显示图形
plt.show()

# 2. 相关系数法
1. 相关系数法用来衡量特征与目标变量之间的线性关系。该方法适用于数值型特征与目标变量的关系分析。相关系数法通常用于特征筛选的早期阶段，以过滤掉与目标变量相关性较低的特征。

2. 原理: 相关系数（如皮尔逊相关系数）度量的是两个变量之间的线性相关性。值的范围从 -1 到 1，分别表示完全负相关和完全正相关，0 则表示无线性相关性。核心公式（皮尔逊相关系数）：其中， 和  分别是特征  和目标变量  的样本值， 和  是它们的均值。

3. 推导：1. 计算均值：首先，计算特征和目标变量的均值。2. 协方差计算：计算特征和目标变量之间的协方差。3. 标准差计算：计算特征  和目标变量  的标准差。4. 相关系数计算：最后，通过协方差和标准差的比值计算相关系数。

4. 选择相关系数较大的特征。

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from scipy.stats import pearsonr

# 设置随机种子以确保结果可复现
np.random.seed(42)

# 生成虚拟数据集
# Ensure 'Target' column is added to the dataframe
df['Target'] = np.random.rand(n_samples)  # Example target values, replace with actual target data

columns = [f'Feature_{i+1}' for i in range(n_features)]
correlations = {}
for col in columns:
    corr, _ = pearsonr(df[col], df['Target'])
    correlations[col] = corr

# 将相关系数结果转换为DataFrame
corr_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation'])

# 绘制图形
plt.figure(figsize=(16, 10))

# 图1：相关系数矩阵的热图
plt.subplot(2, 1, 1)
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation Matrix Heatmap', fontsize=16)

# 图2：特征与目标变量的散点图
plt.subplot(2, 1, 2)
colors = sns.color_palette('husl', n_features)
for i, col in enumerate(columns):
    plt.scatter(df[col], df['Target'], color=colors[i], alpha=0.7, label=f'{col} (Corr: {correlations[col]:.2f})')

plt.xlabel('Feature Value', fontsize=12)
plt.ylabel('Target Value', fontsize=12)
plt.title('Scatter Plot of Features vs. Target', fontsize=16)
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 3. 递归特征消除
递归特征消除（RFE）是一种迭代方法，基于模型的特征重要性逐步减少特征数目。RFE 通过每次训练模型并移除重要性最低的特征，最终找到性能最佳的特征子集。

原理RFE 的核心思想是通过反复训练模型，评估每个特征的重要性，并在每一轮中移除最不重要的特征。最终剩下的特征就是被认为最重要的特征子集。核心公式RFE 不依赖具体的公式，而是基于模型计算的特征重要性得分（如权重或特征贡献度）。例如在线性回归中，使用回归系数的绝对值作为特征的重要性度量。

推导：1. 训练初始模型：使用所有特征训练模型，计算特征的重要性。2. 移除不重要特征：从模型中移除权重绝对值最小的特征（在其他模型中则可能是信息增益、基尼系数等）。3. 递归：重复步骤1和2，直到达到所需的特征数量。4. 最终特征选择：选择在模型性能评估中表现最好的特征子集。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# 生成虚拟数据集
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, n_classes=2, random_state=42)

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 初始化分类器
svc = SVC(kernel="linear")

# 初始化RFE，选择最优特征数
rfe = RFE(estimator=svc, n_features_to_select=5)
rfe.fit(X_train, y_train)

# 绘制特征排序
ranking = rfe.ranking_
plt.figure(figsize=(14, 8))

plt.subplot(1, 2, 1)
plt.bar(range(X.shape[1]), ranking, color='dodgerblue')
plt.xlabel("Feature Index")
plt.ylabel("Ranking")
plt.title("Feature Ranking using RFE")
plt.xticks(range(X.shape[1]), labels=range(1, X.shape[1]+1))

# 计算不同特征数量下的模型性能
scores = []
num_features = range(1, X.shape[1] + 1)  # Adjusted range to match the length of scores
for n in num_features:
    rfe = RFE(estimator=svc, n_features_to_select=n)
    rfe.fit(X_train, y_train)
    score = np.mean(cross_val_score(rfe, X_train, y_train, cv=5))
    scores.append(score)

# 绘制特征数量与模型性能的关系
plt.subplot(1, 2, 2)
plt.plot(num_features, scores, marker='o', color='crimson', linestyle='-', linewidth=2, markersize=8)
plt.xlabel("Number of Selected Features")
plt.ylabel("Cross-Validated Accuracy")
plt.title("Model Performance vs. Number of Features")
plt.grid(True)

# 显示图像
plt.tight_layout()
plt.show()


# 4. L1正则化
1.  Lasso回归是一种线性回归模型，通过在损失函数中加入L1正则化项，促使一些回归系数变为零，从而实现特征选择。Lasso的稀疏性特性使其在处理高维数据时特别有效。

2. 原理: L1正则化会对回归系数施加一个稀疏性约束，使得部分系数被压缩为零。这意味着模型只会使用部分特征，从而实现特征选择的效果。核心公式其中， 是正则化强度， 是特征  的回归系数。

3. 推导：1. 最小化损失函数：目标是找到一组回归系数  ，使得损失函数最小化。2. L1正则化引入：通过引入L1正则化项 ，在回归过程中自动选择特征。3. 梯度下降法：通常使用梯度下降法进行优化。由于L1正则化的非光滑性，可以通过次梯度法或坐标下降法求解。

4. 特征选择：随着的增加，更多的将被压缩为零，从而选择出对模型贡献最大的特征。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 生成虚拟数据集
np.random.seed(0)
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=0)

# 添加一些无关特征
X = np.hstack([X, np.random.randn(X.shape[0], 5)])

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 设置Lasso回归模型
alphas = [0.01, 0.1, 1, 10, 100]
coefs = []
mse_train = []
mse_test = []

# 训练Lasso回归模型并记录系数和均方误差
for alpha in alphas:
    model = Lasso(alpha=alpha, max_iter=10000)
    model.fit(X_train, y_train)
    coefs.append(model.coef_)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse_train.append(mean_squared_error(y_train, y_train_pred))
    mse_test.append(mean_squared_error(y_test, y_test_pred))

# 绘制结果图
plt.figure(figsize=(14, 7))

# 系数图
plt.subplot(1, 2, 1)
for i in range(len(alphas)):
    plt.plot(coefs[i], label=f'Alpha={alphas[i]}', marker='o')
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.title('Lasso Coefficients vs. Feature Index')
plt.legend()
plt.grid(True)

# 均方误差图
plt.subplot(1, 2, 2)
plt.plot(alphas, mse_train, label='Train MSE', marker='o', color='blue')
plt.plot(alphas, mse_test, label='Test MSE', marker='o', color='red')
plt.xscale('log')
plt.xlabel('Alpha (Regularization Strength)')
plt.ylabel('Mean Squared Error')
plt.title('MSE vs. Alpha')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 5. 基于树模型的特征选择

1. 树模型（如决策树、随机森林、XGBoost）可以通过其内部结构自动评估特征的重要性。通过统计每个特征在树中的分裂贡献，生成特征重要性分数，并根据这些分数进行特征选择。

2. 原理: 树模型在构建过程中，通过特征选择来进行节点分裂。某个特征的重要性可以通过其在所有树中的累积贡献来衡量。这种方法对于处理高维数据和非线性关系特别有效。

3. 核心公式特征重要性可以通过累积的分裂点贡献度来计算。对于决策树：其中， 表示特征  的重要性， 是第  个节点由特征  分裂带来的信息增益。

4. 推导：1. 训练树模型：使用训练数据构建树模型，如随机森林或决策树。2. 计算节点分裂信息增益：在每次分裂时，计算信息增益或基尼指数的变化。3. 累积重要性：累积每个特征在所有树中的贡献度，即特征重要性。4. 排序和选择：根据特征重要性得分排序，并选择得分较高的特征。Python实现

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# 生成虚拟数据集
X, y = make_classification(n_samples=500, n_features=20, n_informative=15, n_redundant=5, random_state=42)

# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 使用随机森林分类器进行特征选择
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_scaled, y)

# 获取特征重要性
importances = clf.feature_importances_

# 基于特征重要性选择特征
selector = SelectFromModel(clf, threshold="mean", prefit=True)
X_selected = selector.transform(X_scaled)

# 绘制特征重要性图和选择后特征图

plt.figure(figsize=(14, 6))

# 特征重要性图
plt.subplot(1, 2, 1)
indices = np.argsort(importances)[::-1]
plt.bar(range(X_scaled.shape[1]), importances[indices], color='skyblue', align='center')
plt.xticks(range(X_scaled.shape[1]), indices + 1)
plt.xlabel('Feature Index')
plt.ylabel('Importance')
plt.title('Feature Importance from Random Forest')

# 选择后的特征图
plt.subplot(1, 2, 2)
selected_features = np.sum(selector.get_support())
plt.bar([0], [selected_features], color='salmon', align='center')
plt.xticks([0], ['Selected Features'])
plt.xlabel('Selected Features')
plt.ylabel('Count')
plt.title('Number of Selected Features')

# 显示图形
plt.tight_layout()
plt.show()

# 6. 卡方检验

1. 卡方检验主要用于检测分类特征和分类目标变量之间的独立性。它适合于离散型数据，尤其是在处理分类问题时，可以用来评估每个特征与目标变量的关联性。

2. 原理: 卡方检验计算的是观察频率和期望频率之间的差异，以此判断两个分类变量是否独立。如果独立性假设被拒绝，则说明特征和目标变量之间存在显著关联。核心公式其中， 是第  类的观察频率， 是第  类的期望频率。

3. 推导：1. 构建列联表：根据特征和目标变量的不同取值，构建观测频率表（即列联表）。2. 计算期望频率：其中， 是第  行的总频数， 是第  列的总频数， 是总样本数。 3. 计算卡方统计量：使用卡方公式计算观测频率与期望频率之间的差异。4. 卡方检验：将计算的卡方统计量与卡方分布表对比，确定p值。如果p值小于设定的显著性水平，则拒绝独立性假设，表明该特征对目标变量有显著影响。

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest

# 生成虚拟数据集
np.random.seed(42)
X, y = make_classification(n_samples=1000, n_features=10, n_classes=3, n_informative=5, n_redundant=0, random_state=42)

# 将特征转化为数据框
feature_names = [f'Feature_{i}' for i in range(X.shape[1])]
X_df = pd.DataFrame(X, columns=feature_names)
y_df = pd.Series(y, name='Target')

# 合并特征和目标变量
data = pd.concat([X_df, y_df], axis=1)

# 转换目标变量为类别标签
le = LabelEncoder()
y_encoded = le.fit_transform(y_df)

# 卡方检验
X_new = SelectKBest(chi2, k='all').fit_transform(X, y_encoded)
chi2_values, p_values = chi2(X, y_encoded)

# 创建图形
plt.figure(figsize=(14, 10))

# 特征卡方检验值的条形图
plt.subplot(2, 2, 1)
sns.barplot(x=feature_names, y=chi2_values, palette='viridis')
plt.xticks(rotation=90)
plt.title('Chi-Square Values for Each Feature')
plt.xlabel('Feature')
plt.ylabel('Chi-Square Value')

# 特征p值的条形图
plt.subplot(2, 2, 2)
sns.barplot(x=feature_names, y=p_values, palette='plasma')
plt.xticks(rotation=90)
plt.title('P-Values for Each Feature')
plt.xlabel('Feature')
plt.ylabel('P-Value')

# 特征值分布直方图
plt.subplot(2, 2, 3)
for feature in feature_names:
    sns.histplot(data[feature], kde=True, label=feature, bins=30)
plt.legend()
plt.title('Distribution of Feature Values')
plt.xlabel('Feature Value')
plt.ylabel('Frequency')

# 目标变量的频率分布直方图
plt.subplot(2, 2, 4)
sns.histplot(y_df, kde=True, bins=3, palette='coolwarm')
plt.title('Distribution of Target Variable')
plt.xlabel('Target Class')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


ValueError: Input X must be non-negative.